# Azure AI Agents function calling

This version has the SDK poll on our behalf

## Define a function for your agent to call

In [2]:
import json, os
from typing import Any, Callable, Set, Dict, List, Optional

def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {
        "New York": "Sunny, 25°C", 
        "London": "Cloudy, 18°C", 
        "Tokyo": "Rainy, 22°C"
    }
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

def fetch_restaurant(location: str) -> str:
    """
    Fetches the restaurant information for the specified location.

    :param location (str): The location to fetch the restaurant for.
    :return: Restaurant information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a restaurant API.
    # Here, we'll mock the response.
    mock_restaurant_data = {
        "New York": "Tatiana by Kwame Onwuachi, Katz’s Delicatessen, Peter Luger Steakhouse, Sylvia's, Nathan's Famous", 
        "London": "St. JOHN, Señor Ceviche, Gloria and Circolo Popolare, Normah's, Bouchon Racine", 
        "Tokyo": "Chanko & Wanko Restaurant Asakusa Sumo Club, Sky Restaurant 634 Musashi, Ichiran, Shibuya, Rokkasen Otakibashiidori, Hakushu - Kobe Teppanyaki"
    }
    restaurant = mock_restaurant_data.get(location, "Restaurant data not available for this location.")
    restaurant_json = json.dumps({"restaurant": restaurant})
    return restaurant_json

def fetch_budget() -> str:
    """
    Fetches the budget information for the specified location.
    :return: budget information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a another API.
    # Here, we'll mock the response.
    mock_budget_data = {
        "New York": """
            Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "London": """
            Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
            Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
        """, 
        "Tokyo": """
            Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
            Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
            Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.
        """
    }
    budget_json = json.dumps({"budget": mock_budget_data})
    return budget_json

def fetch_product_info(userquery: str) -> str:
    """
    Fetches the product information for the specified user query.
    :return: product information.
    :rtype: str
    """

    from azure.search.documents import SearchClient
    from azure.search.documents.models import VectorizableTextQuery
    from azure.core.credentials import AzureKeyCredential
    from openai import AzureOpenAI

    azure_search_service_admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
    azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
    azure_search_service_index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
    azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
    azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
    azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

    # Get credential from Azure AI Search Admin key
    credential = AzureKeyCredential(azure_search_service_admin_key)
    search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                                credential=credential, 
                                index_name=azure_search_service_index_name)

    # Azure OpenAI client
    openai_client = AzureOpenAI(
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_endpoint,
        api_key=azure_openai_key)

    # Provide instructions to the model
    SYSTEM_PROMPT="""
    You are an AI assistant that helps users learn from the information found in the source material.
    Answer the query using only the sources provided below.
    Use bullets if the answer has multiple points.
    If the answer is longer than 3 sentences, provide a summary.
    Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
    If there isn't enough information below, say you don't know.
    Do not generate answers that don't use the sources below.
    Query: {query}
    Sources:\n{sources}
    """

    # User Query
    query = userquery 

    # Convert query into vector form
    vector_query = VectorizableTextQuery(text=query, 
                                        k_nearest_neighbors=50, 
                                        fields="text_vector",
                                        weight=1)

    results = search_client.search(
        query_type="semantic", 
        semantic_configuration_name='my-semantic-config',
        search_text=query,
        vector_queries= [vector_query],
        select=["title","chunk"],
        top=5,
    )

    # Use a unique separator to make the sources distinct. 
    # We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
    sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in results])

    response = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": SYSTEM_PROMPT.format(query=query, sources=sources_formatted)
            }
        ],
        model=azure_openai_deployment
    )

    return response.choices[0].message.content


# Statically defined user functions for fast reference
user_functions: Set[Callable[..., Any]] = {
    fetch_weather, fetch_restaurant, fetch_budget, fetch_product_info
}

## STEP 1: Create a client and agent

In [3]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model="gpt-4o", 
    name="my-agent", 
    instructions="""
            You are an AI Travel Agent. 
            You will answer questions about travel based on the tools provided. You will not get information outside of the tools.
            You have access to the following tools:
            - fetch_weather - fetches the weather information for a given location.
            - fetch_restaurant - fetches restaurant information for a given location.
            - fetch_budget - fetches budget information for a given location.
            - fetch_product_info - fetches product information such as travel insurance, luggage, wifi plan, accessories and other products based on user queries.
        """, 
    toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_bY6C6hq6v9L9j88LmJ8qrtsN


## STEP 2: Create a thread

In [4]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

KeyboardInterrupt: 

## Step 3-6: Helper Function
3. Add a message to the thread
4. Run the Agent
5. Check the Run Status
6. Display the Agent's Response


In [4]:
def run_agent(user_input):  
    # Step 3: Add a message to the thread  
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )
    print(f"Created message, ID: {message.id}")

    # Step 4 & 5: Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Step 6: Display the Agent's Response
    elif run.status == 'completed':
            # Fetch all messages in the thread
            messages = project_client.agents.list_messages(thread_id=thread.id)
            if messages.data:
                agent_message = messages.data[0]  # Get the last assistant message
                print(f"Agent Response: {agent_message.content[0].text.value}") 
            else:
                print("No messages found.")

## Running the agent using the Helper Function

In [5]:
user_input = "Hello, what is the weather in Tokyo?"
run_agent(user_input)

Created message, ID: msg_wI7ZVKk7XPrQ9OCIW83vCZ53
Agent Response: The weather in Tokyo is currently rainy, with a temperature of 22°C.


In [6]:
user_input = "What is the budget there?"
run_agent(user_input)

Created message, ID: msg_IejrvsNCM7mVfZPA3bhyNPT6
Agent Response: Here is the budget breakdown for traveling in Tokyo:

- **Budget Travelers**: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
- **Mid-Range Travelers**: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
- **Luxury Travelers**: About $908 per day. This includes luxury hotels, fine dining, and private transportation.


In [7]:
user_input = "Where can I eat in London?"
run_agent(user_input)

Created message, ID: msg_2euFxtzgYFPFaejVC1SBXOli
Agent Response: Here are some great places to eat in London:

- **St. JOHN**: Known for British cuisine.
- **Señor Ceviche**: Specializes in Peruvian dishes.
- **Gloria and Circolo Popolare**: Offers Italian fare with a vibrant atmosphere.
- **Normah's**: Famous for Malaysian home-style cooking.
- **Bouchon Racine**: A delightful French bistro.

Enjoy your meal!


In [8]:
user_input = "I only have 300 USD. What country can I visit for 4 days?"
run_agent(user_input)

Created message, ID: msg_iptCnCBs4t45jGkD3v7wm8bI
Agent Response: Let’s break down the options based on the budgets provided:

With $300 for a 4-day trip:
- **Tokyo ($100/day budget for budget travelers)**: The estimated cost for budget travelers in Tokyo is $100 per day. For 4 days, this would cost $400, which exceeds your $300 budget.

- **London ($75/day for budget travelers)**: The budget for London as a traveler is $75 per day. For 4 days, this totals $300 exactly! London is a viable option.

- **New York ($121/day budget for budget travelers)**: For 4 days, it totals $484, which is again outside your budget.

### Recommendation:
With $300, you can visit **London** and comfortably stay within your budget if you opt for budget accommodations, meals, and public transport!


In [9]:
user_input = "I need a travel lock, travel adapter, and luggage. What can you recommend? Also how much will it cost?"
run_agent(user_input)

Created message, ID: msg_47SrPdHm4qtNmYr50SLE9Uva
Agent Response: Here are the items you requested along with their costs and features:

### 1. **Travel Lock**
- **Product**: WanderSafe Travel Lock by SecureTravel
- **Features**: Durable zinc alloy body, flexible steel cable, TSA-approved, 3-digit resettable combination lock, compact design.
- **Available Colors**: Black, silver, blue, red.
- **Weight**: 0.2 lbs.
- **Price**: $25.
- **Warranty**: 1-year limited warranty covering manufacturing defects.

### 2. **Travel Adapter**
- **Product**: TravelSmart Universal Adapter by GlobeTrek
- **Features**: Universal compatibility for over 150 countries (US, UK, EU, AU plugs), dual USB ports, LED power indicator, built-in fuse.
- **Material**: ABS Plastic.
- **Dimensions**: 2.8 x 2 x 2 inches.
- **Weight**: 0.3 lbs.
- **Price**: $35.
- **Warranty**: 1-year limited warranty.

### 3. **Luggage**
- **Product**: Voyager Pro Luggage by TravelMaster
- **Features**: High-quality polycarbonate, expan

## Delete Agent to free up resources

In [10]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
